# Carbon Investment Potential
Data preparation for the carbon investment potential widget (pie/donut chart).  
Data model is as follows:  

location_id: staging API id  
indicator: intervals + agb + soc + toc   
value:  

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import os


In [24]:
old_carbon = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/mangrove_data_v2_join.csv')
old_carbon.head()

,id,date,gain_m2,loss_m2,length_m,area_m2,hmax_m,agb_mgha_1,hba_m,location_id,...,hba_hist_m,hmax_hist_m,total_carbon,agb_tco2e,bgb_tco2e,soc_tco2e,toc_tco2e,toc_hist_tco2eha,iso,location_type
0,39237,2000-01-01,NaN,NaN,NaN,NaN,11.006700,35.453350,NaN,1105,...,NaN,"[[0,19349.180392156864],[1,19304.67450980392],...",NaN,NaN,NaN,NaN,NaN,NaN,USA,wdpa
1,39238,2007-01-01,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000,NaN,1106,...,NaN,"[[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0],[7,...",NaN,NaN,NaN,NaN,NaN,NaN,VGB,wdpa
2,39239,2008-01-01,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000,NaN,1106,...,NaN,"[[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0],[7,...",NaN,NaN,NaN,NaN,NaN,NaN,VGB,wdpa
3,39365,2000-01-01,NaN,NaN,NaN,NaN,9.389063,54.013480,NaN,1121,...,NaN,"[[0,9122.572549019611],[1,12830.776470588235],...",NaN,NaN,NaN,NaN,NaN,NaN,GNB,wdpa
4,37651,2016-01-01,1.060384e+07,1.328997e+07,691109.54,1.752985e+09,18.773118,178.978127,NaN,907,...,NaN,"[[0,11656],[1,20860.066666666666],[2,0],[3,677...",NaN,5.181886e+07,4.198865e+07,2.858326e+08,3.376514e+08,"{""0--700"":74437,""1400--2100"":1212295,""2100--28...",GAB,country


In [25]:
old_carbon.columns

Index(['id', 'date', 'gain_m2', 'loss_m2', 'length_m', 'area_m2', 'hmax_m',
       'agb_mgha_1', 'hba_m', 'location_id', 'created_at', 'updated_at',
       'con_hotspot_summary_km2', 'net_change_m2', 'agb_hist_mgha_1',
       'hba_hist_m', 'hmax_hist_m', 'total_carbon', 'agb_tco2e', 'bgb_tco2e',
       'soc_tco2e', 'toc_tco2e', 'toc_hist_tco2eha', 'iso', 'location_type'],
      dtype='object')

In [26]:
#check year for Blue Carbon data

old_carbon[old_carbon.toc_hist_tco2eha.notnull()]['date'].value_counts()


2016-01-01    263
Name: date, dtype: int64

In [27]:
old_carbon = old_carbon[old_carbon.toc_hist_tco2eha.notnull()][['location_id', 'agb_tco2e','soc_tco2e','toc_tco2e', 'toc_hist_tco2eha']]
old_carbon.columns = old_carbon.columns.str.replace('_tco2e', '')
old_carbon

,location_id,agb,soc,toc,toc_histha
4,907,5.181886e+07,2.858326e+08,3.376514e+08,"{""0--700"":74437,""1400--2100"":1212295,""2100--28..."
15,893,2.082835e+07,4.880726e+07,6.963561e+07,"{""0--700"":19064,""1400--2100"":247482,""2100--280..."
23,894,2.762126e+06,6.301444e+07,6.577656e+07,"{""0--700"":29051,""1400--2100"":61577,""2100--2800..."
31,895,9.133897e+05,6.336688e+06,7.250078e+06,"{""0--700"":8761,""1400--2100"":21961,""2100--2800""..."
39,896,6.296532e+07,2.565707e+08,3.195360e+08,"{""0--700"":96755,""1400--2100"":1038230,""2100--28..."
...,...,...,...,...,...
2065,1150,0.000000e+00,0.000000e+00,0.000000e+00,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."
2073,1151,1.081230e+04,3.933168e+05,4.041291e+05,"{""0--700"":99,""1400--2100"":600,""2100--2800"":151..."
2081,1152,0.000000e+00,0.000000e+00,0.000000e+00,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."
2089,1153,0.000000e+00,0.000000e+00,0.000000e+00,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."


In [28]:
carb_columns = old_carbon['toc_histha'].map(eval).apply(pd.Series)
carb_columns

,0--700,1400--2100,2100--2800,2800--3500,700--1400
4,74437,1212295,362360,137176,109765
15,19064,247482,132289,2908,191427
23,29051,61577,1,0,608041
31,8761,21961,613,0,45658
39,96755,1038230,479301,222945,347928
...,...,...,...,...,...
2065,0,0,0,0,0
2073,99,600,1513,5,46
2081,0,0,0,0,0
2089,0,0,0,0,0


In [29]:
old_carbon_full = pd.concat([old_carbon[['location_id', 'agb','soc','toc']], carb_columns], axis=1).reset_index().drop(columns = 'index')
old_carbon_full.head()

,location_id,agb,soc,toc,0--700,1400--2100,2100--2800,2800--3500,700--1400
0,907,5.181886e+07,2.858326e+08,3.376514e+08,74437,1212295,362360,137176,109765
1,893,2.082835e+07,4.880726e+07,6.963561e+07,19064,247482,132289,2908,191427
2,894,2.762126e+06,6.301444e+07,6.577656e+07,29051,61577,1,0,608041
3,895,9.133897e+05,6.336688e+06,7.250078e+06,8761,21961,613,0,45658
4,896,6.296532e+07,2.565707e+08,3.195360e+08,96755,1038230,479301,222945,347928


In [30]:
carbon_long = pd.melt(old_carbon_full, id_vars='location_id')
carbon_long['variable'] = carbon_long['variable'].str.replace('--', '-')
carbon_long.rename(columns={'variable':'indicator'}, inplace=True)
carbon_long

,location_id,indicator,value
0,907,agb,5.181886e+07
1,893,agb,2.082835e+07
2,894,agb,2.762126e+06
3,895,agb,9.133897e+05
4,896,agb,6.296532e+07
...,...,...,...
2099,1150,700-1400,0.000000e+00
2100,1151,700-1400,4.600000e+01
2101,1152,700-1400,0.000000e+00
2102,1153,700-1400,0.000000e+00


## Add Locations (old --> new)  
Add ISO code using old location  
Add new location_id using ISO

In [31]:
OLDdataLocation = requests.get('https://mangrove-atlas-api.herokuapp.com/api/v1/locations').json()['data']
#dataLocation = requests.get('http://192.168.50.115:3000/api/v2/locations').json()['data']
old_locations = pd.DataFrame(OLDdataLocation)
old_locations = old_locations[old_locations['location_type'] == 'country']
old_locations = old_locations[['id', 'iso']]
old_locations

,id,iso
3,992,AGO
4,964,ATG
9,993,AUS
11,968,BHR
12,967,BGD
...,...,...
249,988,VUT
250,956,VEN
251,958,VNM
252,957,VGB


In [32]:
df_join = pd.merge(carbon_long, old_locations, left_on='location_id', right_on='id', how = 'inner')
df_join.drop(columns=['id'], inplace=True)
df_join.rename(columns={'location_id':'old_location_id'}, inplace=True)
df_join

,old_location_id,indicator,value,iso
0,907,agb,5.181886e+07,GAB
1,907,soc,2.858326e+08,GAB
2,907,toc,3.376514e+08,GAB
3,907,0-700,7.443700e+04,GAB
4,907,1400-2100,1.212295e+06,GAB
...,...,...,...,...
811,996,0-700,1.541400e+04,DOM
812,996,1400-2100,1.438600e+05,DOM
813,996,2100-2800,2.822400e+04,DOM
814,996,2800-3500,0.000000e+00,DOM


Add new locations

In [33]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [34]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [35]:
df_final = pd.merge(df_join, locations[['iso', 'location_idn']], on='iso', how='left')
df_final = pd.merge(df_final, api_locs, on='location_idn', how='left')
df_final.head()

,old_location_id,indicator,value,iso,location_idn,id
0,907,agb,5.181886e+07,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
1,907,soc,2.858326e+08,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
2,907,toc,3.376514e+08,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
3,907,0-700,7.443700e+04,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
4,907,1400-2100,1.212295e+06,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0


In [36]:
df_final[df_final.id.isnull()]

,old_location_id,indicator,value,iso,location_idn,id
536,962,agb,3.265358e+06,-,NaN,NaN
537,962,soc,1.435122e+07,-,NaN,NaN
538,962,toc,1.761658e+07,-,NaN,NaN
539,962,0-700,4.209000e+03,-,NaN,NaN
540,962,1400-2100,1.179810e+05,-,NaN,NaN
541,962,2100-2800,3.978000e+03,-,NaN,NaN
542,962,2800-3500,0.000000e+00,-,NaN,NaN
543,962,700-1400,8.757000e+03,-,NaN,NaN


In [37]:
df_final.drop(columns=['iso', 'location_idn', 'old_location_id'], inplace=True)
df_final.rename(columns={'id':'location_id'}, inplace=True)
df_final = df_final[df_final.location_id.notnull()]
#df_final['unit'] = 'ha'
df_final['year'] = 2016
df_final.head()

,indicator,value,location_id,year
0,agb,5.181886e+07,3400.0,2016
1,soc,2.858326e+08,3400.0,2016
2,toc,3.376514e+08,3400.0,2016
3,0-700,7.443700e+04,3400.0,2016
4,1400-2100,1.212295e+06,3400.0,2016


In [39]:
df_final.to_csv('../../../../data/OLD_blue_carbon_UPDATE.csv', index=False)